In [1]:
import pyodbc

**Connessione al database con la creazione di due cursori:**
```
- uno per lettura (Eseguire le SELECT) 

- uno per scrittura (Eseguire INSERT, DELETE, ecc...) 
```

In [2]:
connection_params = ("Driver={SQL Server Native Client 11.0};"
            "Server=localhost;"
            "Database=StarTrek_Lannister;"
            "Trusted_Connection=yes;")
read_connection = pyodbc.connect(connection_params)
read_cursor = read_connection.cursor()
write_connection = pyodbc.connect(connection_params)
write_cursor = write_connection.cursor()

**Eliminazione dei record null dalle tabelle** ***raw_movie*** **,** ***raw_series*** **e** ***[title.akas]*** 

In [3]:
write_cursor.execute("delete from raw_movie where title is null")
write_cursor.execute("delete from raw_series where title is null")
write_cursor.execute("delete from [title.akas] where title is null")
write_connection.commit() # memorizzazione delle eliminazioni effettuate (questa istruzione andrà eseguita dopo ogni modifica effettuata al database)

__Modifica dell'involontario incremento automatico avvenuto per la colonna__ ***DS91***

In [4]:
write_cursor.execute("UPDATE raw_series SET series = 'DS9' where series LIKE 'DS%' ")
write_connection.commit()

**Pulizia di tutti i record delle tabelle in cui si andranno a fare inserimenti**

In [5]:
write_cursor.execute("DELETE FROM PersonTheme")
write_cursor.execute("DELETE FROM ProductGenre")
write_cursor.execute("DELETE FROM Theme")
write_cursor.execute("DELETE FROM Person")
write_cursor.execute("DELETE FROM Movie")
write_cursor.execute("DELETE FROM Episode")
write_cursor.execute("DELETE FROM Product")
write_cursor.execute("DELETE FROM Rating")
write_cursor.execute("DELETE FROM Series")
write_connection.commit()

**Modifica della tabella** ***Movie*** **aggiungendo le colonne** ***Foreign_Gross*** **e** ***Worldwide_Gross***

In [6]:
write_cursor.execute("ALTER TABLE Movie ADD Foreign_gross FLOAT")
write_cursor.execute("ALTER TABLE Movie ADD Worldwide_gross FLOAT")
write_connection.commit()

visualizzo raw_movie

In [7]:
vedi = read_cursor.execute("SELECT * FROM raw_movie")
for a in vedi:
    print(a)

('Star Trek: The Motion Picture', '1', '1', '7410.2 – 7414.1', '1979-12-07', '42%', '$82,258,456', '$56,741,544', '$139,000,000', '$35,000,000', '397%', 'TOS', None, None, None, None)
('Star Trek II: The Wrath of Khan', '2', '2', '8130.3 – 8141.6', '1982-06-04', '87%', '$78,912,963', '$18,087,037', '$97,000,000', '$11,000,000', '882%', 'TOS', None, None, None, None)
('Star Trek III: The Search for Spock', '3', '3', '8210.3', '1984-06-01', '80%', '$76,471,046', '$10,528,954', '$87,000,000', '$17,000,000', '512%', 'TOS', None, None, None, None)
('Star Trek IV: The Voyage Home', '4', '4', '8390.0', '1986-11-26', '85%', '$109,713,132', '$23,286,868', '$133,000,000', '$25,000,000', '532%', 'TOS', None, None, None, None)
('Star Trek V: The Final Frontier', '5', '5', '8454.1', '1989-06-09', '22%', '$52,210,049', '$17,999,951', '$70,210,000', '$27,800,000', '253%', 'TOS', None, None, None, None)
('Star Trek VI: The Undiscovered Country', '6', '6', '9521.6 – 9529.1.', '1991-12-06', '81%', '$74,

**Creazione di un dizionario che mantiene in memoria tutti i** ***Series_ID*** **(corrispondenti all'ID degli universi di StarTrek)** **della tabella** ***Series*** **e, nello stesso ciclo, inserimento dei titoli dei film e degli episodi contenuti rispettivamente nelle tabelle** ***raw_movie*** **e** ***raw_series***

In [8]:
series = {}

read_cursor.execute("SELECT DISTINCT series FROM raw_movie WHERE series IS NOT NULL") 
for i, row in enumerate(read_cursor):
    series[row[0]] = i
    write_cursor.execute(f"INSERT INTO Series (Series_Id, Title) VALUES ({i},'{row[0]}')") 

n_series = len(series.keys())
write_connection.commit() 

read_cursor.execute("SELECT DISTINCT series FROM raw_series WHERE series IS NOT Null") 
for i, row in enumerate(read_cursor):
    write_cursor.execute(f"SELECT COUNT(*) FROM Series WHERE Title = '{row[0]}'") 
    n_occurrences = write_cursor.fetchall()
    if n_occurrences[0][0] == 0:
        series[row[0]] = n_series+i
        write_cursor.execute(f"INSERT INTO Series (Series_Id, Title) VALUES ({n_series+i},'{row[0]}')") 
        
write_connection.commit() 

**Modifica della tabella** ***Person*** **andando a cambiare il tipo della colonna in un VARCHAR(50)**

In [9]:
write_cursor.execute("ALTER TABLE Person DROP COLUMN First_name")
write_cursor.execute("ALTER TABLE Person ADD  First_name VARCHAR(50)")
write_connection.commit() 

**Eliminazione della riga dei totali delle colonne nella tabella raw_movie**

In [10]:
write_cursor.execute("delete from raw_movie where Title like '%TOS Totals%'")
write_cursor.execute("delete from raw_movie where Title like '%TNG Totals%'")
write_cursor.execute("delete from raw_movie where Title like '%AR Totals%'")
write_connection.commit()

**Inserimento dei valori provenienti da raw_movie nelle seguenti tabelle:**
```
- Rating
- Product
- Movie
```

In [11]:
read_cursor.execute("SELECT * FROM raw_movie WHERE title IS NOT NULL")
for i, row in enumerate(read_cursor):
    res = float(row[5].replace('%', ''))
    write_cursor.execute(f"INSERT INTO Rating (Rating_Id, Value, Type) VALUES ({i},{res}, 'percentage')")
    write_cursor.execute(f"INSERT INTO Product (Product_Id, Rating_Id, Series_id, Title, Realease_date) VALUES ({i},{i},'{series[row[11]]}','{row[0]}', '{row[4]}')")
    converted_dgross = float(row[6].replace('$', '').replace(',',''))
    converted_fgross = float(row[7].replace('$', '').replace(',',''))
    converted_wgross = float(row[8].replace('$', '').replace(',',''))
    converted_budget = float(row[9].replace('$', '').replace(',',''))
    converted_pgross = float(row[10].replace('%', '').replace(',',''))
        
    write_cursor.execute(f"INSERT INTO Movie (Movie_id, Product_id, Domestic_gross, Foreign_gross, Worldwide_gross, Budget, Profitability_gross, Number, Tos) VALUES('{i}','{i}',{converted_dgross},{converted_fgross},{converted_wgross},{converted_budget},{converted_pgross},{row[1]},{row[2]})")
write_connection.commit()

**Modifica della struttura della tabella** ***Product*** **andando a cambiare il tipo della colonna** ***Rating_ID*** **in un INT che può essere anche NULL**

In [12]:
write_cursor.execute("ALTER TABLE Product ALTER COLUMN Rating_Id INT NULL")
write_connection.commit()

**Inserimento dei valori provenienti da** ***raw_series*** **nelle seguenti tabelle:**
```
- Product
- Episode
```

In [13]:
read_cursor.execute("SELECT COUNT(Product_Id) FROM Product")
n_product=read_cursor.fetchall()[0][0]
read_cursor.execute("SELECT * FROM raw_series WHERE title IS NOT NULL")
for i, row in enumerate(read_cursor):
    somma_prodid = n_product
    if not(i is None):
        somma_prodid += i
    converted_title = row[0].replace("'", '')
    if series.get(row[5]) != None:
        serieId= series.get(row[5])
    else:
        serieId='null'
    write_cursor.execute(f"INSERT INTO Product (Product_Id, Rating_Id, Series_id, Title, Realease_date) VALUES ({somma_prodid}, null, {serieId}, '{converted_title}', '{row[4]}')")
    write_cursor.execute(f"INSERT INTO Episode (Episode_Id, Product_Id, Season, Remastered_release_date) VALUES ({i},{somma_prodid}, '{row[1].split('x')[0]}', '{row[4]}')")   
write_connection.commit()

**Inserimento dei valori provenienti da** ***raw_theme*** **nelle seguenti tabelle:**
```
- Label
- Album
- Product
- Theme
- Person
- PersonTheme
- Genre
- ProductGenre
```

In [14]:
read_cursor.execute("select * from raw_themes")

for i, row in enumerate(read_cursor):
    label_name = row[22]
    write_cursor.execute(f"SELECT Label_ID FROM Label WHERE Name = '{label_name}' ")
    label_id = write_cursor.fetchall()
    if len(label_id) == 0:
        write_cursor.execute("SELECT MAX(Label_ID) FROM Label")
        max_label_id=write_cursor.fetchall()[0][0]
        if max_label_id is None:
            max_label_id = -1
        write_cursor.execute(f"INSERT INTO Label(Label_ID,Name) VALUES ({max_label_id+1}, '{label_name}')")
        write_connection.commit()
        write_cursor.execute(f"SELECT Label_ID FROM Label WHERE Name = '{label_name}' ")
        label_id = write_cursor.fetchall()[0][0]
    else:
        label_id = label_id[0][0]
        
    album_name = row[6]
    album_date = row[7]
    write_cursor.execute(f"SELECT Album_ID FROM Album WHERE Title = '{album_name}' ")
    album_id = write_cursor.fetchall()
    if len(album_id) == 0:
        write_cursor.execute("SELECT MAX(Album_ID) FROM Album")
        max_album_id=write_cursor.fetchall()[0][0]
        if max_album_id is None:
            max_album_id = -1
        write_cursor.execute(f"INSERT INTO Album(Album_ID,Title,Release_date, Label_ID) VALUES ({max_album_id+1},'{album_name[0:49]}','{album_date}', {label_id})")
        write_connection.commit()
        write_cursor.execute(f"SELECT Album_ID FROM Album WHERE Title = '{album_name[0:49]}' ")
        album_id = write_cursor.fetchall()[0][0]
    else:
        album_id = album_id[0][0]
    theme_name = row[1].replace("'", "").replace("(","").replace(")","")[0:49]
    write_cursor.execute("SELECT MAX(Product_ID) FROM Product")
    max_product_id=write_cursor.fetchall()[0][0]
    if max_product_id is None:
        max_product_id = -1
    write_cursor.execute(f"INSERT INTO Product(Product_ID, Title) VALUES ({max_product_id+1},'{theme_name}')")
    write_connection.commit()
    write_cursor.execute(f"INSERT INTO Theme(Theme_ID, Product_ID, Dance,Album_ID) VALUES({i},{max_product_id+1},'{row[9]}', {album_id})")
    write_connection.commit()
    
    persons = row[2].replace("&",",").split(",")
    for person in persons:
        name_components = person.replace("'","").split(" ")
        first_name = name_components[0]
        last_name = name_components[-1]
        middle_name = name_components[1:-1]
        middle_name = " ".join(middle_name)[0:49]
        write_cursor.execute(f"SELECT Person_ID FROM Person WHERE First_name = '{first_name}' AND Middle_name = '{middle_name}' AND Last_name = '{last_name}'")
        person_id = write_cursor.fetchall()
        if len(person_id) == 0:
            write_cursor.execute("SELECT MAX(Person_id) FROM Person")
            max_person_id=write_cursor.fetchall()[0][0]
            if max_person_id is None:
                max_person_id = -1
            write_cursor.execute(f"INSERT INTO Person (Person_id, First_name, Middle_name, Last_name) VALUES ({max_person_id+1}, '{first_name}', '{middle_name}', '{last_name}')")
            write_connection.commit()
            write_cursor.execute(f"SELECT Person_ID FROM Person WHERE First_name = '{first_name}' AND Middle_name = '{middle_name}' AND Last_name = '{last_name}'")
            person_id = write_cursor.fetchall()[0][0]
        else:
            person_id = person_id[0][0]
        write_cursor.execute("SELECT MAX(PersonTheme_ID) FROM PersonTheme")
        max_persontheme_id=write_cursor.fetchall()[0][0]
        if max_persontheme_id is None:
            max_persontheme_id = -1
        write_cursor.execute(f"INSERT INTO PersonTheme(PersonTheme_ID, Theme_ID, Person_ID) VALUES ({max_persontheme_id+1}, {i},{person_id})")
        write_connection.commit()
    
    genres = row[4].replace("&",",").split(",")
    for genre in genres:
        write_cursor.execute(f"SELECT Genre_ID FROM Genre WHERE Name = '{genre}' ")
        genre_id = write_cursor.fetchall()
        if len(genre_id) == 0:
            write_cursor.execute("SELECT MAX(Genre_ID) FROM Genre")
            max_genre_id=write_cursor.fetchall()[0][0]
            if max_genre_id is None:
                max_genre_id = -1
            write_cursor.execute(f"INSERT INTO Genre (Genre_ID, Name) VALUES ({max_genre_id+1}, '{genre[0:49]}')")
            write_connection.commit()
            write_cursor.execute(f"SELECT Genre_ID FROM Genre WHERE Name = '{genre[0:49]}' ")
            genre_id = write_cursor.fetchall()[0][0]
        else:
            genre_id = genre_id[0][0]
        write_cursor.execute("SELECT MAX(ProductGenre_ID) FROM ProductGenre")
        max_productgenre_id=write_cursor.fetchall()[0][0]
        if max_productgenre_id is None:
            max_productgenre_id = -1
        write_cursor.execute(f"INSERT INTO ProductGenre(ProductGenre_ID , Genre_ID, Product_ID) VALUES({max_productgenre_id+1}, {genre_id}, {i})")
        write_connection.commit()

**Inserimento nella tabella** ***Series*** **dei titoli contenuti nella colonna** ***primaryTitle*** **della tabella** ***raw_title_basics*** 

In [15]:
write_cursor.execute("SELECT MAX(Series_ID) FROM Series")
max_series_id = write_cursor.fetchall()[0][0]
write_connection.commit()
read_cursor.execute("select distinct primaryTitle from raw_title_basics")
for i, row in enumerate(read_cursor):
    titolo=row[0].replace("'"," ")
    write_cursor.execute(f"INSERT INTO Series (Series_Id, Title) VALUES ({max_series_id+i+1},'{titolo[0:49]}')")
write_connection.commit()

**Inserimento nella tabella** ***Person*** **delle colonne contenute nella tabella raw** ***name.basics_2*** 

In [16]:
write_cursor.execute("SELECT MAX(Person_ID) FROM Person")
max_person_id = write_cursor.fetchall()[0][0]

read_cursor.execute("SELECT * FROM [name.basics_2]")

for i, row in enumerate(read_cursor):
    name_components = row[1].replace("'","").split(" ")
    first_name = name_components[0]
    last_name = name_components[-1]
    middle_name = name_components[1:-1]
    birth_year = row[2]
    death_year = row[3]
    
    ident_nuovo_persona = max_person_id+i+1
    
    if not birth_year.isnumeric():
        birth_year = "NULL"
    else:
        birth_year = f"'01/01/{birth_year}'"
        
    if not death_year.isnumeric():
        death_year = "NULL"
    else:
        death_year = f"'01/01/{death_year}'"

    if len(middle_name)==0:
        middle_name = "NULL"
    else:
        middle_name = "'"+" ".join(middle_name)+"'"
        
    middle_name = middle_name.replace("'", "").replace("(","").replace(")","").replace('"',"")
    first_name = first_name.replace("'", "").replace("(","").replace(")","").replace('"',"")
    last_name = last_name.replace("'", "").replace("(","").replace(")","").replace('"',"")
    birth_year = birth_year.replace("'", "").replace("(","").replace(")","").replace('"',"")
    death_year = death_year.replace("'", "").replace("(","").replace(")","").replace('"',"")
    
    birth_year = birth_year if birth_year == "NULL" else "'"+birth_year+"'"
    death_year = death_year if death_year == "NULL" else "'"+death_year+"'"
    write_cursor.execute(f"INSERT INTO Person (Person_id, First_name, Middle_name, Last_name, Birth_year, Birth_date) VALUES ({ident_nuovo_persona}, '{first_name[0:49]}', '{middle_name[0:49]}', '{last_name[0:49]}', {birth_year[0:49]}, {death_year[0:49]})")
write_connection.commit()

**Aggiorno la tabella** ***Rating*** **per aggiungere anche il rating associato agli episodi delle serie**

In [17]:
read_cursor.execute("""SELECT primaryTitle, ROUND(SUM(CAST (averageRating AS float)*CAST(numVotes AS float)) / SUM(CAST(numVotes AS float)),1) AS averageRating
FROM [title.ratings] JOIN [title.basics] ON [title.ratings].tconst = [title.basics].tconst 
WHERE [title.basics].primaryTitle IN (SELECT Title FROM raw_series)
GROUP BY primaryTitle""")
write_cursor.execute("SELECT MAX(Rating_ID) FROM Rating")
max_rating_id = write_cursor.fetchall()[0][0]
for i, row in enumerate(read_cursor):
    series_name = row[0].replace("'", "")
    write_cursor.execute(f"SELECT Product_ID FROM Product WHERE Title = '{series_name}'")
    product_id = write_cursor.fetchall()
    if len(product_id)>0:
        product_id = product_id[0][0]
        write_cursor.execute(f"INSERT INTO Rating(Rating_ID, Value, Type) VALUES({max_rating_id+i+1},{row[1]},'0-10')")
        write_connection.commit()
        write_cursor.execute(f"UPDATE Product SET Rating_ID = {max_rating_id+i+1} WHERE Product_ID = {product_id}")
        write_connection.commit()
    

**Chiusura ed eliminazione dei cursori e delle connessioni usati per connettersi al database SQLServer**

In [18]:
read_cursor.close()
del read_cursor
read_connection.close()
del read_connection

write_cursor.close()
del write_cursor
write_connection.close()
del write_connection